In [5]:
import numpy as np 
import pandas as pd 

In [6]:
df = pd.read_csv("./data/Instagram data.csv", encoding="latin1")
df.head()

,Impressions,From Home,From Hashtags,From Explore,From Other,Saves,Comments,Shares,Likes,Profile Visits,Follows,Caption,Hashtags
0,3920,2586,1028,619,56,98,9,5,162,35,2,Here are some of the most important data visua...,#finance #money #business #investing #investme...
1,5394,2727,1838,1174,78,194,7,14,224,48,10,Here are some of the best data science project...,#healthcare #health #covid #data #datascience ...
2,4021,2085,1188,0,533,41,11,1,131,62,12,Learn how to train a machine learning model an...,#data #datascience #dataanalysis #dataanalytic...
3,4528,2700,621,932,73,172,10,7,213,23,8,Heres how you can write a Python program to d...,#python #pythonprogramming #pythonprojects #py...
4,2518,1704,255,279,37,96,5,4,123,8,0,Plotting annotations while visualizing your da...,#datavisualization #datascience #data #dataana...


In [7]:
#Concatenate captions and hashtags in one column
posts_text = df['Caption']+ df['Hashtags'].astype(str)
posts_text = posts_text.str.replace('\xa0', ' ', regex=True)
posts_text[0]


'Here are some of the most important data visualizations that every Financial Data Analyst/Scientist should know.#finance #money #business #investing #investment #trading #stockmarket #data #datascience #dataanalysis #dataanalytics #datascientist #machinelearning #python #pythonprogramming #pythonprojects #pythoncode #artificialintelligence #ai #dataanalyst #amankharwal #thecleverprogrammer'

In [8]:
df['instagram_post'] = posts_text

In [15]:
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import spacy
import nltk
from nltk.corpus import stopwords
import re
import matplotlib.pyplot as plt

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Load data
data = df

# Preprocess the text data
def preprocess_text(text):
    text = re.sub('\s+', ' ', text)  # Remove extra spaces
    text = re.sub('\S*@\S*\s?', '', text)  # Remove emails
    text = re.sub('\'', '', text)  # Remove apostrophes
    text = re.sub('[^a-zA-Z]', ' ', text)  # Remove non-alphabet characters
    text = text.lower()  # Convert to lowercase
    return text

data['instagram_post_cleaned'] = data['instagram_post'].apply(preprocess_text)  

# Tokenize and remove stopwords
def tokenize(text):
    tokens = gensim.utils.simple_preprocess(text, deacc=True)
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

data['tokens'] = data['instagram_post_cleaned'].apply(tokenize)

# Lemmatization using spaCy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def lemmatize(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

data['lemmas'] = data['tokens'].apply(lemmatize)

# Create dictionary and corpus
id2word = corpora.Dictionary(data['lemmas'])
texts = data['lemmas']
corpus = [id2word.doc2bow(text) for text in texts]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=3, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=30,
                                            alpha='auto',
                                            per_word_topics=True)

# Print the topics
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

# Compute coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data['lemmas'], dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


<>:20: DeprecationWarning: invalid escape sequence '\s'
<>:21: DeprecationWarning: invalid escape sequence '\S'
<>:20: DeprecationWarning: invalid escape sequence '\s'
<>:21: DeprecationWarning: invalid escape sequence '\S'
C:\Users\yasmi\AppData\Local\Temp\ipykernel_31436\3546581920.py:20: DeprecationWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text)  # Remove extra spaces
C:\Users\yasmi\AppData\Local\Temp\ipykernel_31436\3546581920.py:21: DeprecationWarning: invalid escape sequence '\S'
  text = re.sub('\S*@\S*\s?', '', text)  # Remove emails
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yasmi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(0, '0.089*"datum" + 0.040*"science" + 0.036*"ai" + 0.035*"datascience" + 0.033*"amankharwal" + 0.033*"datascientist" + 0.032*"dataanalysis" + 0.032*"dataanalytic" + 0.027*"deeplearne" + 0.026*"project"')
(1, '0.076*"python" + 0.047*"pythoncode" + 0.037*"pythonproject" + 0.032*"amankharwal" + 0.030*"use" + 0.023*"programming" + 0.021*"pythondeveloper" + 0.021*"pythonlearne" + 0.020*"code" + 0.018*"language"')
(2, '0.039*"machine" + 0.037*"algorithm" + 0.036*"python" + 0.031*"learning" + 0.028*"learn" + 0.025*"datum" + 0.024*"deeplearning" + 0.024*"datascientist" + 0.024*"dataanalytic" + 0.024*"ai"')

Coherence Score:  0.4240663119720951


In [13]:
topics

[(0,
  '0.083*"datum" + 0.041*"science" + 0.034*"project" + 0.028*"ai" + 0.028*"datascience" + 0.025*"amankharwal" + 0.025*"dataanalysis" + 0.025*"python" + 0.025*"dataanalytic" + 0.024*"machinelearne"'),
 (1,
  '0.077*"python" + 0.036*"programming" + 0.035*"use" + 0.034*"pythoncode" + 0.031*"amankharwal" + 0.024*"code" + 0.022*"pythonproject" + 0.019*"computerscience" + 0.018*"technology" + 0.017*"tech"'),
 (2,
  '0.109*"datum" + 0.049*"datascientist" + 0.049*"ai" + 0.048*"dataanalysis" + 0.047*"datascience" + 0.046*"dataanalytic" + 0.045*"deeplearne" + 0.044*"amankharwal" + 0.043*"science" + 0.039*"machinelearning"'),
 (3,
  '0.051*"python" + 0.044*"machine" + 0.041*"algorithm" + 0.036*"learning" + 0.030*"pythonproject" + 0.030*"pythoncode" + 0.028*"learn" + 0.025*"use" + 0.025*"amankharwal" + 0.023*"datum"'),
 (4,
  '0.056*"python" + 0.041*"pythoncode" + 0.035*"pythonproject" + 0.032*"learn" + 0.029*"ukraine" + 0.027*"use" + 0.026*"sentiment" + 0.026*"amankharwal" + 0.020*"otp" + 0.

In [ ]:
import pyLDAvis.gensim_models

In [16]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.120814  0.122733       1        1  46.555139
1     -0.213011  0.011508       2        1  32.553345
2      0.092197 -0.134241       3        1  20.891516, topic_info=            Term        Freq       Total Category  logprob  loglift
10         datum  178.000000  178.000000  Default  30.0000  30.0000
45       machine   31.000000   31.000000  Default  29.0000  29.0000
37       science   73.000000   73.000000  Default  28.0000  28.0000
88     algorithm   29.000000   29.000000  Default  27.0000  27.0000
179     learning   27.000000   27.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
10         datum   19.742197  178.459437   Topic3  -3.6950  -0.6358
1    amankharwal   16.087735  114.970303   Topic3  -3.8997  -0.4008
21    pythoncode   14.986200  100.370856   Topic3  -3.9706  -0.3359
157         time    7.870187   33.883240   Topic3  -4.6146   0.1060
108     analysis    7.534346   25.104506   Topic3  -4.6582   0.3623

[171 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
0         1  0.760554              ai
0         3  0.229374              ai
88        3  0.967396       algorithm
1         1  0.513176     amankharwal
1         2  0.347916     amankharwal
...     ...       ...             ...
117       2  0.878099             way
194       1  0.103745  webdevelopment
194       2  0.829957  webdevelopment
40        1  0.975591            work
399       3  0.862355         working

[166 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])